# Grain Size Distributions

Figure 6.10 from Chapter 6 of *Interstellar and Intergalactic Medium* by Ryden & Pogge, 2021, 
Cambridge University Press.

Plot of the distribution of dust grain sizes, expressed as volume of dust per logarithmic interval in 
radius a, for graphite grains and PAHs and silicate grains, following
[Weingartner & Draine 2001, ApJ, 548, 296](https://ui.adsabs.harvard.edu/abs/2001ApJ...548..296W/abstract)

This notebook includes a Python port of [grain_dist.f](http://physics.gmu.edu/~joe/files/grain_dist.f) by the
authors, using Fortran code from http://physics.gmu.edu/~joe/sizedists.html.  The Python code with a heavy
Fortran accent.

In [ ]:
%matplotlib inline

import os
import sys
import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, LogLocator, NullFormatter

import warnings
warnings.filterwarnings('ignore',category=UserWarning, append=True)

## Standard Plot Format

Setup the standard plotting format and make the plot. Fonts and resolution adopted follow CUP style.

In [ ]:
figName = 'Fig6_10' 

# graphic aspect ratio = width/height

aspect = 4.0/3.0 # 4:3

# Text width in inches - don't change, this is defined by the print layout

textWidth = 6.0 # inches

# output format and resolution

figFmt = 'png'
dpi = 600

# Graphic dimensions 

plotWidth = dpi*textWidth
plotHeight = plotWidth/aspect
axisFontSize = 10
labelFontSize = 6
lwidth = 0.5
axisPad = 5
wInches = textWidth 
hInches = wInches/aspect

# Plot filename

plotFile = f'{figName}.{figFmt}'

# LaTeX is used throughout for markup of symbols, Times-Roman serif font

plt.rc('text', usetex=True)
plt.rc('font', **{'family':'serif','serif':['Times-Roman'],'weight':'bold','size':'16'})

# Font and line weight defaults for axes

matplotlib.rc('axes',linewidth=lwidth)
matplotlib.rcParams.update({'font.size':axisFontSize})

# axis and label padding

plt.rcParams['xtick.major.pad'] = f'{axisPad}'
plt.rcParams['ytick.major.pad'] = f'{axisPad}'
plt.rcParams['axes.labelpad'] = f'{axisPad}'

## Grain Size Distribution

Python version of grain_dist.f

Based on grain_dist.f by Joe Weingartner, code from

    physics.gmu.edu/~joe/files/grain_dist.f

Calculations presented in [Weingartner & Draine 2001](https://ui.adsabs.harvard.edu/abs/2001ApJ...548..296W/abstract)
equations 4-6, and figures therein.

### Inputs:
<dl>
    <dd>a = grain radius in cm
    <dd>index = integer indicating which conditions to adopt
    <dd>dtype = dust type, 1=silicates, 2=graphite
</dl>

### Returns:

$(1/n_H) dn_{gr}/da$ in cm$^{-1}$.  $n_{gr}(a)$ is the number density of
grains with radius < a, $n_H$ is the H nucleus number density


If invalid index, dtype, or a are given it returns dnda=-1

This program used the version of grain_dist.f with the 2002 Feb 12 bug fix ("Corrected bug--previous version
did not multiply very small carbonaceous grain size dist by b_C").

### Model Indexes

(shifted to 0..N-1 from original Fortran 1..N index):
<pre>
 INDEX    R_V    10^5 b_C   case  
   0      3.1      0.0       A        
   1      3.1      1.0       A       
   2      3.1      2.0       A        
   3      3.1      3.0       A        
   4      3.1      4.0       A        
   5      3.1      5.0       A       
   6      3.1      6.0       A        
   7      4.0      0.0       A
   8      4.0      1.0       A
   9      4.0      2.0       A
   10     4.0      3.0       A
   11     4.0      4.0       A
   12     5.5      0.0       A
   13     5.5      1.0       A
   14     5.5      2.0       A
   15     5.5      3.0       A
   16     4.0      0.0       B
   17     4.0      1.0       B
   18     4.0      2.0       B
   19     4.0      3.0       B
   20     4.0      4.0       B
   21     5.5      0.0       B
   22     5.5      1.0       B
   23     5.5      2.0       B
   24     5.5      3.0       B
</pre>

In [ ]:
def grain_dist(a,index,dtype):
    
    # Data for the different coefficients

    alphagarr = [-2.25,-2.17,-2.04,-1.91,-1.84,-1.72,-1.54,-2.26,-2.16,-2.01,
                  -1.83,-1.64,-2.35,-2.12,-1.94,-1.61,-2.62,-2.52,-2.36,-2.09,
                  -1.96,-2.80,-2.67,-2.45,-1.90]

    betagarr = [-0.0648,-0.0382,-0.111,-0.125,-0.132,-0.322,-0.165,-0.199,
                 -0.0862,-0.0973,-0.175,-0.247,-0.668,-0.67,-0.853,-0.722,
                 -0.0144,-0.0541,-0.0957,-0.193,-0.813,0.0356,0.0129,
                 -0.00132,-0.0517]

    atgarr = [0.00745,0.00373,0.00828,0.00837,0.00898,0.0254,.0107,0.0241,
              0.00867,0.00811,0.0117,0.0152,0.148,0.0686,0.0786,0.0418,0.0187,
              0.0366,0.0305,0.0199,0.0693,0.0203,0.0134,0.0275,0.012]

    acgarr = [0.606,0.586,0.543,0.499,0.489,0.438,0.428,0.861,0.803,0.696,
              0.604,0.536,1.96,1.35,0.921,0.72,5.74,6.65,6.44,4.6,3.48,3.43,
              3.44,5.14,7.28]

    cgarr = [9.94e-11,3.79e-10,5.57e-11,4.15e-11,2.90e-11,3.20e-12,9.99e-12,
             5.47e-12,4.58e-11,3.96e-11,1.42e-11,5.83e-12,4.82e-14,3.65e-13,
             2.57e-13,7.58e-13,6.46e-12,1.08e-12,1.62e-12,4.21e-12,2.95e-13,
             2.74e-12,7.25e-12,8.79e-13,2.86e-12]
    
    alphasarr = [-1.48,-1.46,-1.43,-1.41,-2.1,-2.1,-2.21,-2.03,-2.05,
                  -2.06,-2.08,-2.09,-1.57,-1.57,-1.55,-1.59,-2.01,-2.11,
                  -2.05,-2.1,-2.11,-1.09,-1.14,-1.08,-1.13]

    betasarr = [-9.34,-10.3,-11.7,-11.5,-0.114,-0.0407,0.3,0.668,0.832,0.995,
                 1.29,1.58,1.1,1.25,1.33,2.12,0.894,1.58,1.19,1.64,2.1,-0.37,
                 -0.195,-0.336,-0.109]

    atsarr = [0.172,0.174,0.173,0.171,0.169,0.166,0.164,0.189,0.188,0.185,
              0.184,0.183,0.198,0.197,0.195,0.193,0.198,0.197,0.197,0.198,0.198,
              0.218,0.216,0.216,0.211]

    csarr = [1.02e-12,1.09e-12,1.27e-12,1.33e-12,1.26e-13,1.27e-13,1.e-13,
             5.2e-14,4.81e-14,4.7e-14,4.26e-14,3.94e-14,4.24e-14,4.e-14,
             4.05e-14,3.2e-14,4.95e-14,3.69e-14,4.37e-14,3.63e-14,3.13e-14,
             1.17e-13,1.05e-13,1.17e-13,1.04e-13]
    
    bc5arr = [0.,1.,2.,3.,4.,5.,6.,0.,1.,2.,3.,4.,0.,1.,2.,3.,0.,1.,2.,3.,4.,
              0.,1.,2.,3.]

    # Select coefficients by index
    
    if index < 0 or index > len(alphagarr)-1:  # bad index, return nonsense
        return -1.

    alphag = alphagarr[index]
    betag = betagarr[index]
    atg = atgarr[index]*1.e-4
    acg = acgarr[index]*1.e-4
    cg=cgarr[index]
    alphas = alphasarr[index]
    betas = betasarr[index]
    ats = atsarr[index]*1.e-4
    acs = 1.E-5
    cs = csarr[index]
    bc5 = bc5arr[index]

    # Do the calculation

    if dtype == 1:
        dnda = (cs/a)*math.pow((a/ats),alphas)  # better than ** in this case
        if betas > 0:
            dnda *= (1.+betas*a/ats)
        else:
            dnda /= (1.-betas*a/ats)

        if a > ats:
            dnda *= math.exp(((ats-a)/acs)**3)

    elif dtype == 2:
        dnda = (cg/a)*math.pow((a/atg),alphag)
        if betag > 0:
            dnda *= (1.+betag*a/atg)
        else:
            dnda /= (1.-betag*a/atg)

        if a > atg:
            dnda *= math.exp(((atg-a)/acg)**3)

        a01 = 3.5e-8
        a02 = 3.e-7
        sig = 0.4
        b1  = 2.0496e-7            
        b2  = 9.6005e-11

        dnda_vsg = (b1/a)*math.exp(-0.5*(math.log(a/a01)/sig)**2) + \
            (b2/a)*math.exp(-0.5*(math.log(a/a02)/sig)**2)

        if dnda_vsg >= 0.0001*dnda:
            dnda += bc5*dnda_vsg
    else:
        return -1.  # unknown dtype, return nonsense

    return dnda
    

## Calculate grain size distributions

Calculate carbon (graphite + PAHs) and silicate grain-size distributions.

Draine & Weingartner 2001 Model Index: **6** (Milky Way $R_V$=3.1 and C/H=6x10$^{-5}$.

Range of grain sizes: a = 10nm to 100$\mu$m in steps of 0.01-dex.



In [ ]:
# Using model index 6 (MW Rv=3.1 and C/H=6e-5)
 
i_model = 6

# Logarithmic range of grain sizes in cm(!) in log10 steps

lg_amin = -8.0
lg_amax = -4.0
lg_step = 0.01
numa = 1+int((lg_amax-lg_amin)/lg_step)

lg_a = np.linspace(lg_amin,lg_amax,numa)

amu = []
dnda_C = []
dnda_Si = []
for la in lg_a:
    a = math.pow(10.0,la)
    amu.append(a*10000.0)  # convert to microns
    fac = (4.0*math.pi/3.0)*a**4 # spherical volume factor (V*a*dn/da)
    dnda_C.append(fac*grain_dist(a,i_model,2))
    dnda_Si.append(fac*grain_dist(a,i_model,1))


## Make the Plot

Graphite + PAHs as a solid black line, silicates as a dashed black line.

In [ ]:
# plotting limits

xMin = 3e-4 # microns
xMax = 1.0  # 
yMin = 5.0e-30 # cm^3/H
yMax = 4.0e-27

fig,ax = plt.subplots(figsize=(wInches,hInches),dpi=dpi)

ax.tick_params('both',length=6,width=lwidth,which='major',direction='in',top='on',right='on')
ax.tick_params('both',length=3,width=lwidth,which='minor',direction='in',top='on',right='on')

ax.set_xlim(xMin,xMax)
ax.set_xlabel(r'a [$\mu$m]')
ax.set_xscale('log')

ax.set_ylim(yMin,yMax)
ax.set_yscale('log')
ax.set_ylabel(r'$(4\pi/3){\rm a}^4$ dn/da [cm$^3$/H]')

# Carbonaceous grains (Graphite & PAHs)

ax.plot(amu,dnda_C,'-',color='black',lw=1.2,zorder=10)
ax.text(0.025,1.e-28,r'Graphite \& PAHs',fontsize=axisFontSize,ha='left',color='black')

# Silicates

ax.plot(amu,dnda_Si,'--',color='black',lw=1.2,zorder=10)
ax.text(0.02,4.1e-28,'Silicates',fontsize=axisFontSize,ha='right',color='black')

plt.plot()
plt.savefig(plotFile,bbox_inches='tight',facecolor='white')